In [1]:
import grpc
import tensorflow as tf
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc
from keras_image_helper import create_preprocessor

2023-12-19 11:48:04.372591: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-19 11:48:05.264160: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-19 11:48:05.264301: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-19 11:48:05.439650: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-19 11:48:05.780974: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-19 11:48:05.786546: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [20]:
host = "localhost:8500"
channel = grpc.insecure_channel(host)
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

In [21]:
preprocessor = create_preprocessor("resnet50", target_size=(224, 224))

In [22]:
path = "../illustrations/guess-me.jpg"
x = preprocessor.from_path(path)

In [23]:
def np_to_protobuf(data):
    return tf.make_tensor_proto(data, shape=data.shape)

In [24]:
pb_request = predict_pb2.PredictRequest()
pb_request.model_spec.name = "../api/cat-classifier/"
pb_request.model_spec.signature_name = "serving_default"

pb_request.inputs["input_8"].CopyFrom(np_to_protobuf(x))

In [25]:
pb_response = stub.Predict(pb_request, timeout=20.0)

_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "failed to connect to all addresses; last error: UNKNOWN: ipv4:127.0.0.1:9696: Failed to connect to remote host: Connection refused"
	debug_error_string = "UNKNOWN:Error received from peer  {created_time:"2023-12-19T11:49:04.068532465+01:00", grpc_status:14, grpc_message:"failed to connect to all addresses; last error: UNKNOWN: ipv4:127.0.0.1:9696: Failed to connect to remote host: Connection refused"}"
>

In [ ]:
pb_response

In [ ]:
preds = pb_response.outputs["dense_7"].float_val